In [1]:
from rate import Rate, InterpolationType, StochasticRate
from time_utils.maturity import Maturity
from time_utils.schedule import Schedule
from time_utils.day_count import DayCount
from bonds import Bond, ZCBond
from swaps import Swap, LegType
from convertibles import Convertible
from stocks import Stock

import datetime as dt

In [2]:
# Create some test dates
today = dt.date.today()

# Create maturity points for the yield curve
maturities = {
Maturity(today, today , DayCount.act_365): 0, 
Maturity(today, today + dt.timedelta(days=365*0.5), DayCount.act_365): 0.025,  # 6M: 2.5%
Maturity(today, today + dt.timedelta(days=365), DayCount.act_365): 0.03,  # 1Y: 3%
Maturity(today, today + dt.timedelta(days=365*2), DayCount.act_365): 0.035,  # 2Y: 3.5%
Maturity(today, today + dt.timedelta(days=365*5), DayCount.act_365): 0.042,  # 5Y: 4.2%
Maturity(today, today + dt.timedelta(days=365*10), DayCount.act_365): 0.048  # 10Y: 4.8%
}

# Create rate object with the rate curve
rate_obj = Rate(rate_curve=maturities, interpolation_type=InterpolationType.cubic)


In [3]:
rate_obj._rate_curve[[*rate_obj._rate_curve][-1]]

0.048

In [4]:
stochastic_rate = StochasticRate(rate_curve=maturities, interpolation_type=InterpolationType.linear, num_paths=100)

In [5]:
stochastic_rate.get_curve()

array([0.00013736, 0.00051511, 0.00088908, 0.00114375, 0.00121241,
       0.00170086, 0.00189102, 0.00211776, 0.00229333, 0.00298392,
       0.00336493, 0.00346918, 0.00379877, 0.00395183, 0.0044067 ,
       0.00459048, 0.00495917, 0.00533888, 0.00552521, 0.0058538 ,
       0.00634888, 0.00649487, 0.00660245, 0.00678334, 0.00688292,
       0.00721414, 0.00763047, 0.00817056, 0.00825413, 0.00828503,
       0.00880831, 0.00947297, 0.00976135, 0.01015965, 0.01034222,
       0.01064361, 0.010688  , 0.01110544, 0.01193993, 0.01249083,
       0.01258571, 0.01282443, 0.01320009, 0.01353126, 0.01398161,
       0.01452811, 0.01478993, 0.01490916, 0.01525141, 0.01520329,
       0.01517855, 0.01542581, 0.01593281, 0.01656476, 0.01679226,
       0.01728732, 0.01764708, 0.01783754, 0.01845965, 0.01882573,
       0.01923687, 0.0190774 , 0.01932042, 0.01952344, 0.01978644,
       0.01985391, 0.01975156, 0.01977806, 0.02005695, 0.02053373,
       0.02067555, 0.02079666, 0.02093891, 0.02119365, 0.02138

In [6]:
import matplotlib.pyplot as plt



# Plot the data
plt.figure(figsize=(8, 5))
plt.plot([i.end_date for i in [*stochastic_rate._rate_curve]], stochastic_rate.get_curve(), marker='o', label='Stochastic Rate Curve')
plt.title('Stochastic Rate Curve')
plt.xlabel('Maturity (Years)')
plt.ylabel('Rate')
plt.grid(True)
plt.legend()
plt.show()

In [7]:
list_of_rates = []
for i in stochastic_rate._rate_curve : 
    list_of_rates.append(stochastic_rate._rate_curve[i])

In [14]:
stochastic_rate._rate_curve[0]._rate_curve.keys()

dict_keys([Maturity(start_date=2025-04-26, end_date=2025-04-27, maturity_years=0.0027397260273972603), Maturity(start_date=2025-04-26, end_date=2025-05-10, maturity_years=0.038356164383561646), Maturity(start_date=2025-04-26, end_date=2025-05-24, maturity_years=0.07671232876712329), Maturity(start_date=2025-04-26, end_date=2025-06-08, maturity_years=0.1178082191780822), Maturity(start_date=2025-04-26, end_date=2025-06-22, maturity_years=0.15616438356164383), Maturity(start_date=2025-04-26, end_date=2025-07-07, maturity_years=0.19726027397260273), Maturity(start_date=2025-04-26, end_date=2025-07-21, maturity_years=0.2356164383561644), Maturity(start_date=2025-04-26, end_date=2025-08-05, maturity_years=0.27671232876712326), Maturity(start_date=2025-04-26, end_date=2025-08-19, maturity_years=0.3150684931506849), Maturity(start_date=2025-04-26, end_date=2025-09-03, maturity_years=0.3561643835616438), Maturity(start_date=2025-04-26, end_date=2025-09-17, maturity_years=0.39452054794520547), 

In [20]:
for i in list_rate:
    print(i.get_rate(Maturity(today, today + dt.timedelta(days=365*5))))
print(rate_obj.get_rate(Maturity(today, today + dt.timedelta(days=365*5))))

0.04736393394753159
0.0468984867585583
0.0467764092788026
0.042


In [2]:
zc = ZCBond('zc', rate=Rate(0.05), maturity = 
            Maturity(start_date=dt.datetime(2023, 1, 1),end_date=dt.datetime(2033, 1, 1)), 
            face_value=1000)
zc.get_duration()

10.008219178082191

In [3]:
bond = Bond(
    name="US Treasury Bond",
    rate=Rate(
        rate = 0.06),
    maturity = Maturity(
        start_date=dt.datetime(2023, 1, 1),
        end_date=dt.datetime(2033, 1, 1)),
    face_value = 1000,
    coupon = 0.06,
    nb_coupon=2,
    price=1000
    )

In [4]:
for i in bond.to_zcbonds(): 
    print(i.name, i.price, i.maturity.end_date, i.rate, i.face_value)

US Treasury Bond_ZC_0.5 29.11575898419324 2023-07-02 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_1.0 28.252936007527463 2024-01-01 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_1.5 27.415682121795104 2024-07-02 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_2.0 26.607613101514723 2024-12-31 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_2.5 25.82336167361217 2025-07-01 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_3.0 25.05810634233816 2025-12-31 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_3.5 24.315528760361282 2026-07-02 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_4.0 23.598835831996603 2026-12-31 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_4.5 22.903267213065213 2027-07-01 00:00:00 <rate.Rate object at 0x00000209DC1120D0> 30.0
US Treasury Bond_ZC_5.

In [5]:
bond.get_duration()

7.598439695425409

In [6]:
schedule = Schedule(dt.date(2023, 1, 1), dt.date(2024, 1, 1), frequency=30)

swap = Swap(name="Test Swap", notional=100, 
            schedule=schedule, 
            receiving_leg=LegType.fixed,
            paying_leg=LegType.floating,
            receiving_rate=Rate(0.05),
            paying_rate=Rate(0.04),
            receiving_rate_premium=0,
            paying_rate_premium=0)

In [7]:
swap._get_price()

0.9738702104026713

In [8]:
stock=Stock(
        name="Test Stock",
        price=100,
        div_amount=1,
        div_schedule=Schedule(dt.date(2025, 1, 1), 
                              dt.date(2035, 1, 1), 
                              frequency=180))

stock.div_schedule.schedule

[datetime.date(2025, 1, 1),
 datetime.date(2025, 6, 30),
 datetime.date(2025, 12, 27),
 datetime.date(2026, 6, 25),
 datetime.date(2026, 12, 22),
 datetime.date(2027, 6, 20),
 datetime.date(2027, 12, 17),
 datetime.date(2028, 6, 14),
 datetime.date(2028, 12, 11),
 datetime.date(2029, 6, 9),
 datetime.date(2029, 12, 6),
 datetime.date(2030, 6, 4),
 datetime.date(2030, 12, 1),
 datetime.date(2031, 5, 30),
 datetime.date(2031, 11, 26),
 datetime.date(2032, 5, 24),
 datetime.date(2032, 11, 20),
 datetime.date(2033, 5, 19),
 datetime.date(2033, 11, 15),
 datetime.date(2034, 5, 14),
 datetime.date(2034, 11, 10),
 datetime.date(2035, 1, 1)]

In [9]:
convertible = Convertible(
    name="Test Convertible",
    rate=Rate(0.02),
    maturity=Maturity(
        start_date=dt.date(2025, 1, 1),
        end_date=dt.date(2035, 1, 1)),
    face_value=100,
    coupon=0.05,
    nb_coupon=2,
    conversion_ratio=1.2,
    stock=Stock(
        name="Test Stock",
        price=100,
        div_amount=1,
        div_schedule=Schedule(dt.date(2025, 1, 1), 
                              dt.date(2035, 1, 1), 
                              frequency=180),
        volatility=0.2),
)

In [10]:
convertible.bond_component

126.95521243913494

In [11]:
convertible.option_component

np.float64(40.296208803359576)

In [12]:
convertible.price

np.float64(167.25142124249453)

In [13]:
convertible._get_bond_component_duration()

np.float64(6.265283997308113)

In [14]:
convertible.get_duration()

0.04175844812691167


np.float64(5.1611948820996085)